In [ ]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Data/winequality-red-cleaned.csv', sep=',')

In [3]:
df.head()

,volatile acidity,residual sugar,chlorides,total sulfur dioxide,pH,sulphates,alcohol,quality,category
0,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246,5,Bad
1,1.967442,0.043416,0.223875,0.624363,-0.719933,0.128950,-0.584777,5,Bad
2,1.297065,-0.169427,0.096353,0.229047,-0.331177,-0.048089,-0.584777,5,Bad
3,-1.384443,-0.453218,-0.264960,0.411500,-0.979104,-0.461180,-0.584777,6,Good
4,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246,5,Bad


In [88]:
y = pd.DataFrame([0. if item == 'Good' else 1. for item in df['category']])
y = [0. if item == 'Good' else 1. for item in df['category']]
y[:5]

[1.0, 1.0, 1.0, 0.0, 1.0]

In [5]:
features_df = df.drop(['quality', 'category'], axis=1)

In [6]:
features_df.head()

,volatile acidity,residual sugar,chlorides,total sulfur dioxide,pH,sulphates,alcohol
0,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246
1,1.967442,0.043416,0.223875,0.624363,-0.719933,0.128950,-0.584777
2,1.297065,-0.169427,0.096353,0.229047,-0.331177,-0.048089,-0.584777
3,-1.384443,-0.453218,-0.264960,0.411500,-0.979104,-0.461180,-0.584777
4,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246


In [7]:
import torch

In [8]:
# x = torch.from_numpy(features_df.as_matrix())

In [9]:
# y = torch.from_numpy(np.array(y))

In [89]:
# TODO: pip install sklearn
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_df, y, test_size=0.2, random_state=42)
# X_train = features_df
# y_train = np.array(y)

In [85]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(7, 2)

    def forward(self, x):
        x = self.fc1(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()

In [69]:
learning_rate = 0.05
batch_size = 126
epochs = 1000
epochs_to_print = epochs / 10

In [70]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss() # use a Classification Cross-Entropy loss
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

In [98]:
from torch.autograd import Variable

for epoch in range(epochs): # loop over the dataset multiple times
    
    running_loss = 0.0
    for start, end in zip(range(0, len(X_train), batch_size), 
                          range(batch_size, len(X_train), batch_size)):
        # get the inputs
        
        inputs = torch.from_numpy(X_train[start:end].as_matrix())
        inputs = inputs.float()
        labels = torch.Tensor(y_train[start:end])
        labels = labels.long()

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()        
        optimizer.step()
        
        # print statistics
        running_loss += loss.data[0]
        if epoch % epochs_to_print == 0:
            print(loss.data[0])
            print('%d loss: %.3f' % (epoch+1, running_loss / epochs_to_print))
            running_loss = 0.0
print('Finished Training')

0.7050687074661255
1 loss: 0.007
0.8019934296607971
1 loss: 0.008
0.81717848777771
1 loss: 0.008
0.80559241771698
1 loss: 0.008
0.832951009273529
1 loss: 0.008
0.7637039422988892
1 loss: 0.008
0.7856991291046143
1 loss: 0.008
0.8053922653198242
1 loss: 0.008
0.8317679166793823
1 loss: 0.008
0.7746209502220154
1 loss: 0.008
0.7050687074661255
101 loss: 0.007
0.8019934296607971
101 loss: 0.008
0.81717848777771
101 loss: 0.008
0.80559241771698
101 loss: 0.008
0.832951009273529
101 loss: 0.008
0.7637039422988892
101 loss: 0.008
0.7856991291046143
101 loss: 0.008
0.8053922653198242
101 loss: 0.008
0.8317679166793823
101 loss: 0.008
0.7746209502220154
101 loss: 0.008
0.7050687074661255
201 loss: 0.007
0.8019934296607971
201 loss: 0.008
0.81717848777771
201 loss: 0.008
0.80559241771698
201 loss: 0.008
0.832951009273529
201 loss: 0.008
0.7637039422988892
201 loss: 0.008
0.7856991291046143
201 loss: 0.008
0.8053922653198242
201 loss: 0.008
0.8317679166793823
201 loss: 0.008
0.7746209502220154
2

In [94]:
outputs = net(Variable(torch.from_numpy(X_test.as_matrix()).float()))

In [95]:
outputs

Variable containing:
 0.2278 -0.2686
-0.0934 -0.6461
 0.9509  0.2293
-0.0093 -0.3103
 0.2688  0.3197
-0.3921 -0.3179
 0.4485 -1.2844
 0.0185  0.4320
 0.6816  0.0087
 0.3159  0.8576
 1.0107  0.1433
 0.1086  0.9090
 0.2555 -0.0023
 0.1596 -0.0837
-0.1149 -0.3554
 0.6805  0.4540
 0.1972 -0.3665
 0.2784 -0.4089
 1.2121  0.2237
 1.1209 -0.3195
-0.5688  0.3195
 0.0874 -0.1692
 0.4145  1.0045
 0.7105  0.5407
 0.1598  0.3867
-0.0036  0.4986
 0.1450 -0.3110
 0.0064  0.3585
-0.5806 -0.9479
 0.1719 -0.0043
 0.1338 -0.3316
-0.3272  0.0113
 0.3421 -0.0204
 0.5787 -0.1128
 0.2828  0.4691
 0.1444 -0.8405
 0.7067  0.7682
 0.5418  0.9353
 1.1372 -0.1941
 0.5233  0.4342
 0.8480 -0.5082
 0.2070 -0.8048
 0.8991 -0.2977
 0.8100 -1.0490
 1.0235 -0.1778
 1.2606  0.6733
 1.1835  0.3641
 0.6185 -0.6355
 0.7252  0.0990
 0.4243  0.4551
 0.2078 -1.1969
 1.7513 -0.6638
 0.5134  0.6934
 1.5184  0.8100
-0.1867  0.0899
 0.5852 -0.8730
 0.9173  0.7369
-0.3425 -0.3347
 0.1169  0.2352
-0.0619  0.7319
 0.1984 -0.4466
 0.